In [1]:
import tensorflow as tf

Colab Env

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!cp "/content/drive/MyDrive/dataset.zip" "/content/"
!unzip /content/dataset.zip

train_data_folder="/content/dataset_train"
eval_data_folder="/content/dataset_eval"

Local Env

In [2]:
train_data_folder="./dataset_train"
eval_data_folder="./dataset_eval"

In [3]:
image_size = (224,224)
batch_size = 128

In [5]:
#import data:

train_dataset = tf.keras.preprocessing.image_dataset_from_directory(train_data_folder,image_size=image_size,shuffle=True,batch_size=batch_size)
class_names = train_dataset.class_names
print("Classes :", class_names)
class_number = len(class_names)
print(class_number)

eval_dataset = tf.keras.preprocessing.image_dataset_from_directory(eval_data_folder,image_size=image_size,shuffle=False,batch_size=batch_size,class_names=class_names)
#test_dataset = tf.keras.preprocessing.image_dataset_from_directory(test_data_folder,image_size=image_size,shuffle=True,batch_size=batch_size)

Found 345623 files belonging to 26 classes.
Classes : ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
26
Found 2600 files belonging to 26 classes.


In [6]:
import tensorflow as tf

AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.prefetch(AUTOTUNE)
eval_dataset   = eval_dataset.prefetch(AUTOTUNE)

class RandomInvert(tf.keras.layers.Layer):
    def __init__(self, p=0.5, **kwargs):
        super().__init__(**kwargs)
        self.p = p

    def call(self, x, training=None):
        training = tf.constant(False) if training is None else tf.cast(training, tf.bool)

        def do_aug():
            batch = tf.shape(x)[0]
            mask = tf.random.uniform([batch, 1, 1, 1]) < self.p
            return tf.where(mask, 1.0 - x, x)

        return tf.cond(training, do_aug, lambda: x)

aug = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.05),
    tf.keras.layers.RandomContrast(0.10),
    tf.keras.layers.RandomTranslation(0.05, 0.05),
    tf.keras.layers.RandomZoom(0.07),
    tf.keras.layers.RandomBrightness(factor=0.2, value_range=(0, 1)),
    tf.keras.layers.GaussianNoise(0.03),
    RandomInvert(0.5),
], name="augment")

In [7]:
def conv_block(c, k=3, s=1):
    return tf.keras.Sequential([
        tf.keras.layers.Conv2D(c, k, s, padding="same", use_bias=False),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.ReLU()
    ])

In [8]:
inputs = tf.keras.Input(shape=image_size + (3,))
x = tf.keras.layers.Rescaling(1./255)(inputs)
x = aug(x)


x = conv_block(48)(x)
x = conv_block(48)(x); x = conv_block(96, s=2)(x)
x = conv_block(96)(x); x = conv_block(192, s=2)(x)
x = conv_block(192)(x); x = conv_block(384, s=2)(x)


x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.4)(x)
outputs = tf.keras.layers.Dense(class_number, activation="softmax")(x)
model = tf.keras.Model(inputs, outputs)

model.compile(
    optimizer=tf.keras.optimizers.AdamW(3e-4, weight_decay=1e-4),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)



In [9]:
cbs = [
    tf.keras.callbacks.ModelCheckpoint(
        "best_scratch.keras", save_best_only=True, monitor="val_accuracy"
    ),
    tf.keras.callbacks.ReduceLROnPlateau(patience=3, factor=0.5,monitor="val_accuracy"),
    tf.keras.callbacks.EarlyStopping(patience=8, restore_best_weights=True, monitor="val_accuracy"),
]

In [ ]:
model.fit(
    train_dataset,
    validation_data=eval_dataset,
    epochs=150,
    callbacks=cbs
)

Epoch 1/150
2701/2701 ━━━━━━━━━━━━━━━━━━━━ 1539s 563ms/step - accuracy: 0.1819 - loss: 2.7908 - val_accuracy: 0.0835 - val_loss: 3.5340 - learning_rate: 3.0000e-04
Epoch 2/150
2701/2701 ━━━━━━━━━━━━━━━━━━━━ 1527s 565ms/step - accuracy: 0.5281 - loss: 1.5827 - val_accuracy: 0.1442 - val_loss: 3.2811 - learning_rate: 3.0000e-04
Epoch 3/150
2701/2701 ━━━━━━━━━━━━━━━━━━━━ 1583s 586ms/step - accuracy: 0.6638 - loss: 1.1343 - val_accuracy: 0.2012 - val_loss: 2.9607 - learning_rate: 3.0000e-04
Epoch 4/150
2701/2701 ━━━━━━━━━━━━━━━━━━━━ 1683s 623ms/step - accuracy: 0.7294 - loss: 0.9128 - val_accuracy: 0.2996 - val_loss: 2.4479 - learning_rate: 3.0000e-04
Epoch 5/150
2701/2701 ━━━━━━━━━━━━━━━━━━━━ 1730s 640ms/step - accuracy: 0.7739 - loss: 0.7650 - val_accuracy: 0.3500 - val_loss: 2.2963 - learning_rate: 3.0000e-04
Epoch 6/150
2701/2701 ━━━━━━━━━━━━━━━━━━━━ 1729s 640ms/step - accuracy: 0.8022 - loss: 0.6649 - val_accuracy: 0.4769 - val_loss: 1.7870 - learning_rate: 3.0000e-04
Epoch 7/150
2701

In [ ]:
model.evaluate(eval_dataset) # or test_dataset if defined

In [ ]:
model.save("v1_handsign.keras")